### requirement

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# requirements
!pip install openai==0.28.1
!pip install llmx==0.0.15a0
!pip install cohere
!pip install tiktoken


In [ ]:
import openai
# fine tuning을 위한 개인키
openai.api_key = ''
key = openai.api_key
print(type(openai.api_key))
print(type(openai))

- 약 200개의 데이터를 작성을 하고 이후 HyperClovaX를 활용해 데이터증강을 실시했습니다. 아래의 코드는 200여개의 데이터를 생성하고 Fine tuning을 진행한 코드입니다.

In [ ]:
import json
from sklearn.model_selection import train_test_split

with open("/content/drive/MyDrive/언어지능_챗봇/output_data3.jsonl", "r", encoding = "utf-8") as file:
    data = [json.loads(line) for line in file]


train_data, validation_data = train_test_split(data, test_size=0.2, random_state=42)


with open("/content/drive/MyDrive/언어지능_챗봇/train_data2.jsonl", "w", encoding="utf-8") as train_file:
    for item in train_data:
        train_file.write(json.dumps(item, ensure_ascii=False) + "\n")

# val data
with open("/content/drive/MyDrive/언어지능_챗봇/validation_data2.jsonl", "w", encoding="utf-8") as validation_file:
    for item in validation_data:
        validation_file.write(json.dumps(item, ensure_ascii=False) + "\n")



In [ ]:
train_file_id = openai.File.create(
    file=open("/content/drive/MyDrive/언어지능_챗봇/train_data2.jsonl", "rb"),
    purpose='fine-tune'
    )['id']

val_file_id = openai.File.create(
    file=open("/content/drive/MyDrive/언어지능_챗봇/validation_data2.jsonl", "rb"),
    purpose='fine-tune'
)['id']

In [ ]:
fine_tuning_job = openai.FineTuningJob.create(
    model="gpt-3.5-turbo",
    training_file=train_file_id,
    validation_file = val_file_id
)

print(fine_tuning_job)

In [ ]:
# user 질문
qs = str(input())
response = openai.ChatCompletion.create(
    model = "ft:gpt-3.5-turbo-1106:personal::8ckuzUTf",
    messages = [
        {"role": "system", "content": "너는 식단을 평가해주는 챗봇이야"},
        {"role": "user", "content": qs}
    ]
)

result = response["choices"][0]["message"]["content"]
messages0 = []

# 일반 질문
if result == "assistant: 0":

    content0 = qs
    messages0.append({"role":"user", "content":content0})


    response0 = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = messages0
    )

    chat_response = response0.choices[0].message.content
    print(f'Diet ChatBot: {chat_response}')
    messages0.append({"role":"assistant", "content": chat_response})

# 식단 평가에 관한 질문
else:
    print("DB에 접근해서 유저의 데이터를 불러와야 합니다.")


- 이후 HyperCLovaX를 통해 데이터를 증강하고 약 1만여개의 데이터를 생성해 최종 Fine Tuning한 model을 만들었습니다.

In [ ]:
import pandas as pd
import json

# 마지막 csv파일
csv_file_path = '/content/drive/MyDrive/언어지능_챗봇/chatbot_learning_data(final).csv'

df = pd.read_csv(csv_file_path, encoding='cp949')

json_data = []

# csv to json
for index, row in df.iterrows():
    user_content = row["Text"].replace("'", "\"").replace('User: ', '')
    assistant_content = row["Completion"].replace("'", "\"").replace('Assistant: ', '')

    message = {
        "role": "system",
        "content": '너는 식단을 평가해주는 챗봇이야',
    }
    user_message = {
        "role": "user",
        "content": user_content,
    }
    assistant_message = {
        "role": "assistant",
        "content": assistant_content,
    }

    # message를 json 데이터에 추가함.
    json_data.append({"messages": [message, user_message, assistant_message]})

# '을 "로 바꾸는 작업
for i in range(len(json_data)):
    for key, value in json_data[i].items():
        if isinstance(value, list):
            for item in value:
                for k, v in item.items():
                    if isinstance(v, str):
                        item[k] = v.replace("'", '"')

# jsonl 파일로 저장
json_file_path = '/content/drive/MyDrive/언어지능_챗봇/chatbot_learning_data(final).jsonl'
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    for data in json_data:
        json_file.write(json.dumps(data, ensure_ascii=False) + "\n")



### data to train & validation

In [ ]:
import json
from sklearn.model_selection import train_test_split

with open("/content/drive/MyDrive/언어지능_챗봇/chatbot_learning_data(final).jsonl", "r", encoding = "utf-8") as file:
    data = [json.loads(line) for line in file]


train_data, validation_data = train_test_split(data, test_size=0.2, random_state=42)


with open("/content/drive/MyDrive/언어지능_챗봇/final_train_data.jsonl", "w", encoding="utf-8") as train_file:
    for item in train_data:
        train_file.write(json.dumps(item, ensure_ascii=False) + "\n")

# val data
with open("/content/drive/MyDrive/언어지능_챗봇/final_validation_data.jsonl", "w", encoding="utf-8") as validation_file:
    for item in validation_data:
        validation_file.write(json.dumps(item, ensure_ascii=False) + "\n")



In [ ]:
print(data[0])
print(train_data[0])
print(validation_data[0])


- train과 validation으로 나눠진 데이터를 바탕으로 openai에서 gpt-3.5-turbo-1106 model을 활용해 Fine Tuning을 진행했습니다